In [1]:
import pandas as pd

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader

In [3]:
from utils_training import Dataset_FV
from utils_training import FVTrainingData
from utils_training import evaluate_FVmodel

In [4]:
from behavior_model import FVBehaviorModel

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
# Replace 'your_file.csv' with the actual path to your CSV file
file_path = 'Synthetic_crash_scenarios.csv'
# file_path = '/content/drive/MyDrive/Colab Notebooks/Thesis Crash Generation/Synthetic_crash_scenarios.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

## Build the dataset

In [7]:
acc_min, acc_max, acc_interval =  -9.5, 4, 0.25
acc_std_dev = 0.07
Inputs, Inputs_Vl, Outputs, Weights = FVTrainingData(df)

In [8]:
## Load the train, val, test indexes

import pickle
with open('.\\utils_data\\Idx_02_split.pkl', 'rb') as f:
    list_train_val_test = pickle.load(f)
list_train_val_test = list_train_val_test[0]

Idx_train, Idx_val, Idx_test = list_train_val_test[0], list_train_val_test[1], list_train_val_test[2]

In [9]:
## Split the Inputs, Outputs, Weights based on train, val, test indexes

Inputs_train = [Inputs[i] for i in Idx_train]
Inputs_Vl_train = [Inputs_Vl[i] for i in Idx_train]
Outputs_train = [Outputs[i] for i in Idx_train]
Weights_train = [Weights[i] for i in Idx_train]

Inputs_val = [Inputs[i] for i in Idx_val]
Inputs_Vl_val = [Inputs_Vl[i] for i in Idx_val]
Outputs_val = [Outputs[i] for i in Idx_val]
Weights_val = [Weights[i] for i in Idx_val]

In [10]:
## Create dataset and dataloader

input_size = Inputs[0].shape[1] # Dimension of the input
batch_size = 64

# dataset = BCDataset(Inputs, Outputs, Weights)
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

dataset_train = Dataset_FV(Inputs_train, Inputs_Vl_train, Outputs_train, Weights_train)
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

dataset_val = Dataset_FV(Inputs_val, Inputs_Vl_val, Outputs_val, Weights_val)
dataloader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)

## Model Training

In [11]:
## Network Parameters 
gru_hidden_size = 256 # Size of GRU's hidden layer
mlp_hidden_size = 128 # Size of MLP's hidden layer
output_size = int((acc_max - acc_min)/acc_interval) + 1

## Training Parameters
num_epochs = 500
learning_rate = 0.0001

## Instantiate model, loss function, and optimizer
model = FVBehaviorModel(input_size, gru_hidden_size, mlp_hidden_size, output_size).to(device)
# criterion = nn.KLDivLoss(reduction='batchmean').to(device)
criterion = nn.KLDivLoss(reduction='none').to(device)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

In [12]:
best_val_loss = float('inf')
best_train_loss = float('inf')
best_trainval_loss = float('inf')
best_val_epoch = 0
best_train_epoch = 0
best_trainval_epoch = 0
best_val_model_path = f'.\\weights\\bc_fv_val_02.pth'
best_train_model_path = f'.\\weights\\bc_fv_train_02.pth'
best_trainval_model_path = f'.\\weights\\bc_fv_trainval_02.pth'

In [13]:
num_epochs = 4000

In [14]:
for epoch in range(num_epochs):
    model.train() # set model as training mode
    for inputs, inputs_vl, targets, sample_weights in dataloader_train:
        inputs = inputs.float().to(device)
        inputs_vl = inputs_vl.reshape(inputs_vl.shape[0],1,inputs_vl.shape[1])
        inputs_vl = inputs_vl.float().to(device)
        targets = targets.float().to(device)
        sample_weights = sample_weights.to(device)
        
        optimizer.zero_grad()
        log_probs = model(inputs,inputs_vl)
        ## calculate loss
        loss = criterion(log_probs, targets)
        weighted_loss = torch.sum(torch.sum(loss,axis=2) * sample_weights.reshape(-1,1))/torch.sum(sample_weights) ## loss per weight
        weighted_loss.backward()
        optimizer.step()
    
    ## Calculate the mean, min, max loss of this epoch
    model.eval()  # set model as evaluation mode
    ## loss of training dataset
    train_loss_mean, train_loss_min, train_loss_max = evaluate_FVmodel(model, criterion, dataloader_train, device)
    ## loss of validation dataset
    test_loss_mean, test_loss_min, test_loss_max = evaluate_FVmodel(model, criterion, dataloader_val, device)
    ## loss of combination
    trainval_loss_mean = (test_loss_mean*len(dataset_val) + train_loss_mean*len(dataset_train)) / (len(dataset_val)+len(dataset_train))

    print(f'Epoch {epoch+1}/{num_epochs}, Loss_mean: {round(train_loss_mean.item(),2)}, \
     Loss_min: {round(train_loss_min.item(),2)}, Loss_max: {round(train_loss_max.item(),2)} \n \
     Loss_mean_val: {round(test_loss_mean.item(),2)}, Loss_min_val: {round(test_loss_min.item(),2)}, \
     Loss_max_val: {round(test_loss_max.item(),2)} \n \
     Loss_mean_trainval: {round(trainval_loss_mean.item(),2)}')

    # Save the model if the current test loss is the best seen so far
    if test_loss_mean < best_val_loss:
        best_val_epoch = epoch+1
        best_val_loss = test_loss_mean
        best_val_model = model.state_dict()
    if train_loss_mean < best_train_loss:
        best_train_epoch = epoch+1
        best_train_loss = train_loss_mean
        best_train_model = model.state_dict()
    if trainval_loss_mean < best_trainval_loss:
        best_trainval_epoch = epoch+1
        best_trainval_loss = trainval_loss_mean
        best_trainval_model = model.state_dict()
        
torch.save(best_val_model, best_val_model_path)
print(f"Model saved with the best val loss. \n Epoch: {best_val_epoch}, Loss_val: {round(best_val_loss.item(),2)}")
torch.save(best_train_model, best_train_model_path)
print(f"Model saved with the best train loss. \n Epoch: {best_train_epoch}, Loss_val: {round(best_train_loss.item(),2)}")
torch.save(best_trainval_model, best_trainval_model_path)
print(f"Model saved with the best train loss. \n Epoch: {best_trainval_epoch}, Loss_val: {round(best_trainval_loss.item(),2)}")

Epoch 1/4000, Loss_mean: 369.59,      Loss_min: 355.14, Loss_max: 385.81 
      Loss_mean_val: 364.97, Loss_min_val: 352.33,      Loss_max_val: 376.26 
      Loss_mean_trainval: 365.19
Epoch 2/4000, Loss_mean: 324.53,      Loss_min: 282.16, Loss_max: 347.17 
      Loss_mean_val: 349.69, Loss_min_val: 325.66,      Loss_max_val: 369.26 
      Loss_mean_trainval: 348.49
Epoch 3/4000, Loss_mean: 329.72,      Loss_min: 327.49, Loss_max: 332.03 
      Loss_mean_val: 336.53, Loss_min_val: 296.21,      Loss_max_val: 367.57 
      Loss_mean_trainval: 336.21
Epoch 4/4000, Loss_mean: 338.23,      Loss_min: 304.26, Loss_max: 379.07 
      Loss_mean_val: 325.53, Loss_min_val: 280.42,      Loss_max_val: 355.85 
      Loss_mean_trainval: 326.13
Epoch 5/4000, Loss_mean: 306.41,      Loss_min: 298.13, Loss_max: 315.0 
      Loss_mean_val: 315.5, Loss_min_val: 259.16,      Loss_max_val: 348.82 
      Loss_mean_trainval: 315.06
Epoch 6/4000, Loss_mean: 287.3,      Loss_min: 268.82, Loss_max: 307.36 
    

Epoch 46/4000, Loss_mean: 165.85,      Loss_min: 146.93, Loss_max: 181.67 
      Loss_mean_val: 177.57, Loss_min_val: 114.1,      Loss_max_val: 253.79 
      Loss_mean_trainval: 177.01
Epoch 47/4000, Loss_mean: 179.49,      Loss_min: 155.58, Loss_max: 221.44 
      Loss_mean_val: 174.89, Loss_min_val: 107.64,      Loss_max_val: 238.67 
      Loss_mean_trainval: 175.11
Epoch 48/4000, Loss_mean: 139.98,      Loss_min: 104.84, Loss_max: 158.54 
      Loss_mean_val: 173.42, Loss_min_val: 109.91,      Loss_max_val: 257.51 
      Loss_mean_trainval: 171.83
Epoch 49/4000, Loss_mean: 247.62,      Loss_min: 122.1, Loss_max: 433.9 
      Loss_mean_val: 173.12, Loss_min_val: 110.57,      Loss_max_val: 290.98 
      Loss_mean_trainval: 176.67
Epoch 50/4000, Loss_mean: 160.03,      Loss_min: 149.52, Loss_max: 170.65 
      Loss_mean_val: 169.53, Loss_min_val: 109.58,      Loss_max_val: 246.99 
      Loss_mean_trainval: 169.08
Epoch 51/4000, Loss_mean: 157.05,      Loss_min: 147.65, Loss_max: 163.85

Epoch 91/4000, Loss_mean: 107.32,      Loss_min: 43.08, Loss_max: 159.3 
      Loss_mean_val: 154.25, Loss_min_val: 83.29,      Loss_max_val: 241.49 
      Loss_mean_trainval: 152.01
Epoch 92/4000, Loss_mean: 120.96,      Loss_min: 95.74, Loss_max: 137.91 
      Loss_mean_val: 155.63, Loss_min_val: 93.32,      Loss_max_val: 219.69 
      Loss_mean_trainval: 153.98
Epoch 93/4000, Loss_mean: 141.67,      Loss_min: 111.01, Loss_max: 165.25 
      Loss_mean_val: 157.45, Loss_min_val: 88.39,      Loss_max_val: 262.02 
      Loss_mean_trainval: 156.7
Epoch 94/4000, Loss_mean: 153.18,      Loss_min: 127.5, Loss_max: 194.62 
      Loss_mean_val: 153.76, Loss_min_val: 88.94,      Loss_max_val: 221.82 
      Loss_mean_trainval: 153.73
Epoch 95/4000, Loss_mean: 139.1,      Loss_min: 123.61, Loss_max: 150.77 
      Loss_mean_val: 152.95, Loss_min_val: 101.64,      Loss_max_val: 236.03 
      Loss_mean_trainval: 152.29
Epoch 96/4000, Loss_mean: 123.29,      Loss_min: 105.76, Loss_max: 134.57 
     

Epoch 136/4000, Loss_mean: 127.69,      Loss_min: 113.44, Loss_max: 141.39 
      Loss_mean_val: 147.04, Loss_min_val: 98.75,      Loss_max_val: 206.86 
      Loss_mean_trainval: 146.11
Epoch 137/4000, Loss_mean: 101.48,      Loss_min: 58.72, Loss_max: 141.25 
      Loss_mean_val: 144.77, Loss_min_val: 72.02,      Loss_max_val: 224.3 
      Loss_mean_trainval: 142.71
Epoch 138/4000, Loss_mean: 130.42,      Loss_min: 116.8, Loss_max: 156.19 
      Loss_mean_val: 145.71, Loss_min_val: 92.82,      Loss_max_val: 216.86 
      Loss_mean_trainval: 144.99
Epoch 139/4000, Loss_mean: 102.42,      Loss_min: 64.86, Loss_max: 135.46 
      Loss_mean_val: 144.83, Loss_min_val: 88.98,      Loss_max_val: 208.69 
      Loss_mean_trainval: 142.81
Epoch 140/4000, Loss_mean: 85.2,      Loss_min: 17.71, Loss_max: 123.04 
      Loss_mean_val: 144.89, Loss_min_val: 74.96,      Loss_max_val: 237.95 
      Loss_mean_trainval: 142.04
Epoch 141/4000, Loss_mean: 153.83,      Loss_min: 113.51, Loss_max: 224.96 
 

Epoch 181/4000, Loss_mean: 123.75,      Loss_min: 99.06, Loss_max: 154.41 
      Loss_mean_val: 137.08, Loss_min_val: 95.05,      Loss_max_val: 189.44 
      Loss_mean_trainval: 136.45
Epoch 182/4000, Loss_mean: 110.84,      Loss_min: 106.96, Loss_max: 114.57 
      Loss_mean_val: 135.93, Loss_min_val: 87.9,      Loss_max_val: 185.79 
      Loss_mean_trainval: 134.73
Epoch 183/4000, Loss_mean: 107.13,      Loss_min: 100.66, Loss_max: 112.58 
      Loss_mean_val: 137.24, Loss_min_val: 76.71,      Loss_max_val: 211.2 
      Loss_mean_trainval: 135.81
Epoch 184/4000, Loss_mean: 96.15,      Loss_min: 68.15, Loss_max: 116.17 
      Loss_mean_val: 138.63, Loss_min_val: 82.37,      Loss_max_val: 219.82 
      Loss_mean_trainval: 136.6
Epoch 185/4000, Loss_mean: 127.2,      Loss_min: 106.81, Loss_max: 161.78 
      Loss_mean_val: 136.51, Loss_min_val: 83.97,      Loss_max_val: 193.74 
      Loss_mean_trainval: 136.07
Epoch 186/4000, Loss_mean: 127.59,      Loss_min: 96.28, Loss_max: 164.33 
  

...

Epoch 3802/4000, Loss_mean: 23.03,      Loss_min: 12.76, Loss_max: 43.19 
      Loss_mean_val: 97.13, Loss_min_val: 27.83,      Loss_max_val: 227.42 
      Loss_mean_trainval: 93.6
Epoch 3803/4000, Loss_mean: 30.86,      Loss_min: 12.06, Loss_max: 67.11 
      Loss_mean_val: 99.72, Loss_min_val: 25.5,      Loss_max_val: 233.43 
      Loss_mean_trainval: 96.43
Epoch 3804/4000, Loss_mean: 14.62,      Loss_min: 13.41, Loss_max: 16.54 
      Loss_mean_val: 102.34, Loss_min_val: 24.91,      Loss_max_val: 286.6 
      Loss_mean_trainval: 98.16
Epoch 3805/4000, Loss_mean: 18.98,      Loss_min: 13.45, Loss_max: 27.99 
      Loss_mean_val: 103.27, Loss_min_val: 31.96,      Loss_max_val: 329.29 
      Loss_mean_trainval: 99.25
Epoch 3806/4000, Loss_mean: 30.86,      Loss_min: 12.98, Loss_max: 64.04 
      Loss_mean_val: 102.72, Loss_min_val: 39.59,      Loss_max_val: 282.63 
      Loss_mean_trainval: 99.29
Epoch 3807/4000, Loss_mean: 10.86,      Loss_min: 5.72, Loss_max: 16.21 
      Loss_mean_v

Epoch 3848/4000, Loss_mean: 13.6,      Loss_min: 11.61, Loss_max: 16.6 
      Loss_mean_val: 100.57, Loss_min_val: 40.69,      Loss_max_val: 342.74 
      Loss_mean_trainval: 96.42
Epoch 3849/4000, Loss_mean: 11.16,      Loss_min: 9.15, Loss_max: 13.64 
      Loss_mean_val: 96.79, Loss_min_val: 45.7,      Loss_max_val: 176.71 
      Loss_mean_trainval: 92.71
Epoch 3850/4000, Loss_mean: 20.03,      Loss_min: 12.19, Loss_max: 35.66 
      Loss_mean_val: 102.12, Loss_min_val: 58.97,      Loss_max_val: 247.4 
      Loss_mean_trainval: 98.21
Epoch 3851/4000, Loss_mean: 9.71,      Loss_min: 2.82, Loss_max: 15.12 
      Loss_mean_val: 102.2, Loss_min_val: 34.86,      Loss_max_val: 273.66 
      Loss_mean_trainval: 97.79
Epoch 3852/4000, Loss_mean: 25.39,      Loss_min: 11.56, Loss_max: 51.73 
      Loss_mean_val: 99.49, Loss_min_val: 44.63,      Loss_max_val: 229.23 
      Loss_mean_trainval: 95.96
Epoch 3853/4000, Loss_mean: 21.98,      Loss_min: 10.72, Loss_max: 41.28 
      Loss_mean_val: 

Epoch 3894/4000, Loss_mean: 18.43,      Loss_min: 11.37, Loss_max: 30.47 
      Loss_mean_val: 97.29, Loss_min_val: 36.03,      Loss_max_val: 292.58 
      Loss_mean_trainval: 93.53
Epoch 3895/4000, Loss_mean: 18.94,      Loss_min: 9.59, Loss_max: 32.99 
      Loss_mean_val: 97.55, Loss_min_val: 32.5,      Loss_max_val: 259.16 
      Loss_mean_trainval: 93.8
Epoch 3896/4000, Loss_mean: 16.91,      Loss_min: 11.78, Loss_max: 26.47 
      Loss_mean_val: 99.48, Loss_min_val: 48.44,      Loss_max_val: 213.73 
      Loss_mean_trainval: 95.54
Epoch 3897/4000, Loss_mean: 13.06,      Loss_min: 11.58, Loss_max: 15.89 
      Loss_mean_val: 103.76, Loss_min_val: 38.61,      Loss_max_val: 334.25 
      Loss_mean_trainval: 99.44
Epoch 3898/4000, Loss_mean: 20.75,      Loss_min: 13.02, Loss_max: 35.67 
      Loss_mean_val: 105.91, Loss_min_val: 50.69,      Loss_max_val: 240.54 
      Loss_mean_trainval: 101.84
Epoch 3899/4000, Loss_mean: 8.93,      Loss_min: 1.29, Loss_max: 14.89 
      Loss_mean_va

Epoch 3940/4000, Loss_mean: 12.29,      Loss_min: 11.23, Loss_max: 12.97 
      Loss_mean_val: 101.4, Loss_min_val: 35.03,      Loss_max_val: 197.7 
      Loss_mean_trainval: 97.15
Epoch 3941/4000, Loss_mean: 11.6,      Loss_min: 9.33, Loss_max: 13.98 
      Loss_mean_val: 101.82, Loss_min_val: 36.63,      Loss_max_val: 199.61 
      Loss_mean_trainval: 97.52
Epoch 3942/4000, Loss_mean: 11.21,      Loss_min: 8.89, Loss_max: 13.22 
      Loss_mean_val: 98.84, Loss_min_val: 48.21,      Loss_max_val: 258.66 
      Loss_mean_trainval: 94.66
Epoch 3943/4000, Loss_mean: 29.59,      Loss_min: 11.0, Loss_max: 63.16 
      Loss_mean_val: 97.93, Loss_min_val: 33.53,      Loss_max_val: 219.11 
      Loss_mean_trainval: 94.67
Epoch 3944/4000, Loss_mean: 22.28,      Loss_min: 11.7, Loss_max: 42.25 
      Loss_mean_val: 97.91, Loss_min_val: 41.7,      Loss_max_val: 219.22 
      Loss_mean_trainval: 94.31
Epoch 3945/4000, Loss_mean: 19.59,      Loss_min: 11.59, Loss_max: 34.7 
      Loss_mean_val: 98

Epoch 3986/4000, Loss_mean: 9.81,      Loss_min: 5.12, Loss_max: 13.17 
      Loss_mean_val: 97.46, Loss_min_val: 43.86,      Loss_max_val: 224.52 
      Loss_mean_trainval: 93.28
Epoch 3987/4000, Loss_mean: 8.35,      Loss_min: 0.11, Loss_max: 12.62 
      Loss_mean_val: 97.59, Loss_min_val: 40.15,      Loss_max_val: 264.35 
      Loss_mean_trainval: 93.34
Epoch 3988/4000, Loss_mean: 14.76,      Loss_min: 12.19, Loss_max: 19.12 
      Loss_mean_val: 97.05, Loss_min_val: 44.41,      Loss_max_val: 209.89 
      Loss_mean_trainval: 93.13
Epoch 3989/4000, Loss_mean: 11.6,      Loss_min: 8.04, Loss_max: 13.56 
      Loss_mean_val: 98.54, Loss_min_val: 46.06,      Loss_max_val: 222.71 
      Loss_mean_trainval: 94.39
Epoch 3990/4000, Loss_mean: 8.93,      Loss_min: 0.27, Loss_max: 14.75 
      Loss_mean_val: 99.02, Loss_min_val: 34.44,      Loss_max_val: 250.92 
      Loss_mean_trainval: 94.72
Epoch 3991/4000, Loss_mean: 11.12,      Loss_min: 6.95, Loss_max: 14.57 
      Loss_mean_val: 102.